In [2]:
filename = "TUD-Stadtmitte-raw.webm"
frame_width, frame_height = 1280,720
# modelPath = "./yolo-head-detection.pt"
modelPath = "./yolov8s.pt"
fps = 25

## BOTSORT

In [9]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT
from pathlib import Path
frame_width, frame_height = 1280,720
model = YOLO(modelPath,verbose=False)

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green|
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture(f"./input_videos/{filename}")
# cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = BoTSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False,track_buffer=fps*15,frame_rate=fps,fuse_first_associate=True)
thickness = 2
color = (0,0,255)
# cap_out = cv2.VideoWriter(f"output_videos/botsort_{filename}", cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
#                           (frame.shape[1], frame.shape[0]))
count = 0
id_set = set()
while(cap.isOpened()):
    count +=1 
    detections = model.predict(frame,verbose=False)[0]
    # results = []
    if not ret:
        break

    # if np.array(detections.boxes.data.tolist()).ndim < 2:
    #     results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0 or conf < 0.7:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Frame: {count}',
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            
            cv2.imshow('Frame', frame)

            # cap_out.write(frame)

    except Exception as e:
        #  print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 
print(f"botsort_{filename}")
print(len(id_set),id_set)

2024-02-16 19:07:52.139 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2024-02-16 19:07:52.139 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


[[    0.95491           0           0           0           0           0           0]
 [          0     0.94086           0           0           0           0           0]
 [          0           0           0           0      0.8601           0      0.3336]
 [          0           0           0     0.78398           0           0           0]
 [          0   0.0088545     0.91671           0           0    0.043708           0]
 [          0           0    0.047689           0           0     0.95424           0]]
[[          0           0           0           0     0.93203           0           0]
 [     0.9476           0           0           0           0           0           0]
 [          0           0           0     0.81856           0           0     0.32922]
 [          0     0.78278           0           0           0           0           0]
 [          0           0     0.97879           0           0     0.05853           0]
 [          0           0    0.057838     

## BotSORT TEMP

In [ ]:
# import cv2
# from ultralytics import YOLO
# import matplotlib.pyplot as plt
# import numpy as np
# from boxmot import BoTSORT
# from pathlib import Path
# frame_width, frame_height = 1280,720
# # model = YOLO("yolo-head-detection.pt")
# model = YOLO(modelPath)


# def calculate_color_based_on_id(id):
#   colors = [[255, 0, 0], # Red
#           [0, 255, 0], # Green|
#           [0, 0, 255], # Blue
#           [255, 255, 0], # Yellow
#           [255, 0, 255], # Magenta
#           [0, 255, 255], # Cyan
#           [128, 128, 128], # Gray
#           [255, 255, 255], # White
#           [0, 0, 0], # Black
#           [192, 192, 192]]
#   return colors[id % len(colors) ]



# # cap = cv2.VideoCapture(f"./input_videos/{filename}")
# cap = cv2.VideoCapture(1)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
# ret, frame = cap.read() 
# tracker = BoTSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False,with_reid=True)
# thickness = 2
# color = (0,0,255)
# # cap_out = cv2.VideoWriter(f"output_videos/botsort_{filename}", cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
# #                           (frame.shape[1], frame.shape[0]))

# while(cap.isOpened()):
#     id_set = set()
#     detections = model.predict(frame,verbose=False)[0]
#     results = []
#     if not ret:
#         break

#     if np.array(detections.boxes.data.tolist()).ndim < 2:
#         results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
#     try:
#         ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
#         xyxys = ts[:,0:4].astype('int') 
#         ids = ts[:, 4].astype('int')  
#         confs = ts[:, 5]
#         clss = ts[:, 6]
#         if ts.shape[0] != 0:
#             for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

#                 if cls != 0:
#                     continue
#                 id_set.add(id)
#                 cv2.rectangle(
#                     frame,
#                     (xyxy[0], xyxy[1]),
#                     (xyxy[2], xyxy[3]),
#                     calculate_color_based_on_id(id),
#                     thickness
#                 )
#                 cv2.putText(
#                     frame,
#                     f'id: {id}',
#                     (xyxy[0], xyxy[1]-10),
#                     cv2.FONT_HERSHEY_SIMPLEX,
#                     0.5,
#                     calculate_color_based_on_id(id),
#                     thickness
#                 )
#             print(len(id_set))
#             cv2.putText(
#                     frame,
#                     f'Count: {len(id_set)}',
#                     (10, 10),
#                     cv2.FONT_HERSHEY_SIMPLEX,
#                     0.5,
#                     (0,0,0),
#                     thickness
#                 )
#             cv2.imshow('Frame', frame)
#             # cap_out.write(frame)

#     except Exception as e:
#          print("Error:",e)
#          pass
#     ret, frame = cap.read()         
#     if cv2.waitKey(25) & 0xFF == ord('q'): 
#             break

# cap.release() 
# cv2.destroyAllWindows() 
# print(f"botsort_{filename}")

## StrongSort

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT,StrongSORT
from pathlib import Path
model = YOLO(modelPath)

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture(f"./input_videos/{filename}")
# cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = StrongSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False)
thickness = 2
color = (0,0,255)
id_set = set()
while(cap.isOpened()):
    detections = model.predict(frame,verbose=False)[0]
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.imshow('Frame', frame)
    except Exception as e:
         print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 
print(len(id_set),id_set)

## ByteTrack

In [ ]:
import cv2
import argparse

from ultralytics import YOLO
import supervision as sv


# cap = cv2.VideoCapture(f"./input_videos/{filename}")
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)


# model = YOLO('yolo-head-detection.pt')
model = YOLO(modelPath)

CLASS_NAMES_DICT = model.model.names
box_annotator = sv.BoxAnnotator()
# mask_annotator = sv.MaskAnnotator()
tracker = sv.ByteTrack(frame_rate=30,track_buffer=120)
print(model.model.names)

ret,frame = cap.read()
cap_out = cv2.VideoWriter(f"out.mp4", cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
                      (frame.shape[1], frame.shape[0]))
while ret:
    ids = set()
    result = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)  
    detections = detections[(detections.class_id==0)]
    detections = tracker.update_with_detections(detections=detections)
    labels  = []
    for _, _, confidence, class_id, tracker_id in detections:
        labels.append(f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}")
        ids.add(tracker_id)
    
    annotated_image = box_annotator.annotate(frame, detections=detections,labels=labels)
    # annotated_image = mask_annotator.annotate(scene=frame,detections=detections,opacity=0.5)
    cv2.putText(
                    annotated_image,
                    f"Count {len(ids)}",
                    (10, 20),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    2
                )
    cv2.imshow("YoloV8",annotated_image)
    cap_out.write(annotated_image)
    # ESC -> 27
    if cv2.waitKey(30) == 27:
        break
    ret,frame = cap.read()


# Deep SORT

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT,StrongSORT,DeepOCSORT
from pathlib import Path
model = YOLO(modelPath)

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture(f"./input_videos/{filename}")
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = DeepOCSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False)
thickness = 2
color = (0,0,255)
id_set = set()
while(cap.isOpened()):
    detections = model.predict(frame,verbose=False)[0]
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.imshow('Frame', frame)
    except Exception as e:
         print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 
print(len(id_set),id_set)